## Steven Miller
### DSC 650 Winter 2020
### 2020-01-08

#### Exercise 5.2: Create a Small Data Warehouse

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
warehouse_dir = '/home/steven/Documents/DSC 650 Week 5/data'
import os

spark = SparkSession.builder.appName('Exercise5').config("spark.sql.warehouse.dir", warehouse_dir).getOrCreate()

#### **1. Gazetteer Data**

**a. Create Unmanaged Tables**

The first step of this assignment involves loading the data from the CSV files, combining the file with the file for the other year, and saving it to disk as a table. The following code should provide a template to help you combine tables and save them to the warehouse directory. Click on the image to download the sample code.

In [2]:
tables = [_[:-4] for _ in os.listdir('gazetteer/2018/')]

for table in tables:
    csv_file_path1 = f'gazetteer/2017/{table}.csv'
    csv_file_path2 = f'gazetteer/2018/{table}.csv'

    df1 = spark.read.load(csv_file_path1, format='csv', sep=',', inferSchema=True, header=True)
    df2 = spark.read.load(csv_file_path2, format='csv', sep=',', inferSchema=True, header=True)

    df = df1.unionAll(df2)
    df.write.saveAsTable(table)

For each CSV file in the 2017 and 2018 directories, load the data into Spark, combine it with the corresponding data from the other year and save it to disk. Once you have finished saving all of the files as tables, verify that you have loaded the files properly by loading the tables into Spark, and performing a simple row count on each table.

The following Python code should provide you a template for loading the tables as an external table in Spark. Click on the image to download the sample code.

In [3]:
def create_external_table(table_name):
    table_dir = os.path.join(warehouse_dir, table_name)
    return spark.catalog.createExternalTable(table_name, table_dir)

def create_external_tables():
    for table_name in tables:
        create_external_table(table_name)


As stated previously, in a typical Hadoop distribution, you could save these tables as persistent tables in Apache Hive, but since we are not introducing Hive in this class, we need to load these tables into Spark and query them using SQL within Python.

The following code shows how to count the number of rows in the places table and show the results. Click on the image to download the code.

As an aside, spark.catalog module offers useful utility functions such as spark.catalog.listTables() to list all of the currently available tables. These are useful for inspecting the Spark SQL warehouse.

In [4]:
spark.sql("SELECT COUNT(*) AS row_count FROM places").show()

+---------+
|row_count|
+---------+
|    59151|
+---------+



**b. Load and Query Tables**

Now that we have saved the data to external tables, we will load the tables back into Spark and create a report using Spark SQL. For this report, we will create a report on school districts for the states of Nebraska and Iowa using the elementary_schools, secondary_schools and unified_school_districts tables. Using Spark SQL, create a report with the following information.

This table contains the number of elementary, secondary, and unified school districts in each state for each year. Note that the numbers in this table are notional and do not represent the actual results.

In [5]:
spark.sql("""SELECT u.state, u.year, e.Elementary, s.Secondary, u.Unified  FROM 
                  (SELECT state, year, COUNT(name) as Unified
                  FROM unified_school_districts
                  GROUP BY state, year) u
                  FULL OUTER JOIN
                  (SELECT state, year, COUNT(name) as Elementary
                  FROM elementary_schools
                  GROUP BY state, year) e ON u.state=e.state AND u.year=e.year
                  FULL OUTER JOIN
                  (SELECT state, year, COUNT(name) as Secondary
                  FROM secondary_schools
                  GROUP BY state, year) s ON u.state=s.state AND e.state=s.state AND u.year=s.year AND e.year=s.year
              WHERE u.state='NE' OR u.state='IA'
              ORDER BY u.state, u.year
              """).na.fill(0).show()

+-----+----+----------+---------+-------+
|state|year|Elementary|Secondary|Unified|
+-----+----+----------+---------+-------+
|   IA|2017|         0|        0|    336|
|   IA|2018|         0|        0|    333|
|   NE|2017|         0|        0|    251|
|   NE|2018|         0|        0|    246|
+-----+----+----------+---------+-------+



#### **2. Flight Data**

In the previous exercise, you joined data from flights and airport codes to create a report. Create an external table for airport_codes and domestic_flights from the domestic-flights/flights.parquet and airport-codes/airport-codes.csv files. Recreate the report of top ten airports for 2008 using Spark SQL instead of dataframes.



In [6]:
df_flights = spark.read.parquet('domestic-flights/flights.parquet')
df_airport_codes = spark.read.load('airport-codes/airport-codes.csv', format='csv', sep=',', inferSchema=True, header=True)
df_flights.write.saveAsTable('flights')
df_airport_codes.write.saveAsTable('airport_codes')

In [7]:
spark.sql('''SELECT RANK() OVER (ORDER BY subtable.total_passengers DESC) as `Rank`, a.name as `Airport Name`, a.iata_code as `IATA Code`, 
            subtable.total_passengers as `Total Passengers`, INT(subtable.daily_passengers) as `Daily Passengers`, subtable.total_flights as `Total Flights`, 
            INT(subtable.daily_flights)  as `Daily Flights` FROM airport_codes a
            INNER JOIN
            (SELECT f.destination_airport_code as dest_code, SUM(f.passengers) as total_passengers, SUM(f.passengers)/365 as daily_passengers, SUM(f.flights) as total_flights, SUM(f.flights)/365 as daily_flights
            FROM flights f
            LEFT OUTER JOIN airport_codes d ON d.iata_code = f.destination_airport_code
            WHERE f.flight_year = 2008
            GROUP BY f.destination_airport_code) subtable
            ON a.iata_code = subtable.dest_code
            ORDER BY total_passengers DESC
            LIMIT 10
            ''').show()

+----+--------------------+---------+----------------+----------------+-------------+-------------+
|Rank|        Airport Name|IATA Code|Total Passengers|Daily Passengers|Total Flights|Daily Flights|
+----+--------------------+---------+----------------+----------------+-------------+-------------+
|   1|Hartsfield Jackso...|      ATL|        35561795|           97429|       395192|         1082|
|   2|Chicago O'Hare In...|      ORD|        26398793|           72325|       356570|          976|
|   3|Dallas Fort Worth...|      DFW|        22883558|           62694|       270243|          740|
|   4|Los Angeles Inter...|      LAX|        19741782|           54087|       215000|          589|
|   5|McCarran Internat...|      LAS|        18262263|           50033|       164123|          449|
|   6|Phoenix Sky Harbo...|      PHX|        17305718|           47412|       181259|          496|
|   7|Charlotte Douglas...|      CLT|        15038489|           41201|       205040|          561|
